In [1]:
########################################################################################################################
# This script performs winsorization (also known as clipping) and scaling of feature values for each partition, each 
# experiment setting, and each level of data-granularity.
########################################################################################################################

In [2]:
########################################################################################################################
# Import packages
########################################################################################################################
import numpy as np
import os
import pandas as pd
import warnings
from ast import literal_eval
from datetime import datetime
from itertools import product, chain
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
####################################################################################################################
# USER_SPECIFIC SETTING
# DICT_PATH: Path of the helper file Variable_Type_Categorization.xlsx 
# (downloadable from this repo at 02_Python_Scripts/05_Data_Dictionaries/)
# FEAT_IN_DIR_PATH: Path of the input directory of the feature datasets 
# (created in P03a_Feature_Extraction_Patient.ipynb and P03b_Feature_Extraction_Encounter.ipynb)
####################################################################################################################
DICT_PATH: str = '../00_Data/99_Dictionary/Variable_Type_Categorization.xlsx'
FEAT_IN_DIR_PATH: str = '../00_Data/02_Processed_Data/Features/'

In [ ]:
########################################################################################################################
# USER-SPECIFIC SETTING
# Cs: Different numbers of feature encounteres to be included
# Ds: Different maximum widths of the look-back window in days
########################################################################################################################
Cs : list[int] = [1, 2, 3, 4]
Ds : list[int] = [60, 120, 180]

In [ ]:
########################################################################################################################
# USER-SPECIFIC SETTING
# SIGMA_MULTIPLIER: Number of standard deviations (sigma) away from the mean (mu). Values of continuous variables 
# outside the range of [mu - SIGMA_MULTIPLE * sigma, mu + SIGMA_MULTIPLE * sigma] will be clipped to the mentioned range.
########################################################################################################################
SIGMA_MULTIPLIER: float = 3.0

In [ ]:
########################################################################################################################
# Define the granularity and partition of the datasets
########################################################################################################################
granular_list: list[str] = ['Patient', 'Encounter']
partitions: list[str] = ['train', 'test']

In [ ]:
########################################################################################################################
# Define a function to compare the number of cells in two data matrices in terms of number of differences
########################################################################################################################
def diff_count(A, B):
    l0 = np.sum(np.not_equal(A, B), axis=0)
    l1 = np.sum(np.logical_and(np.isnan(A), np.isnan(B)), axis=0)
    l = l0 - l1
    l_count = np.sum(l > 0)
    l_pct = l_count * 100 / len(l) 
    l_v_count = np.sum(l)
    l_v_pct = l_v_count * 100 / np.sum(~np.isnan(A))
    return l_count, l_pct, l_v_count, l_v_pct

In [ ]:
########################################################################################################################
# Load the data dictionary
#######################################################################################################################
df_pat_dict: pd.DataFrame = pd.read_excel(DICT_PATH, sheet_name='Patient')
df_enc_dict: pd.DataFrame = pd.read_excel(DICT_PATH, sheet_name='Encounter')
dfs_dict: dict[str, pd.DataFrame] = {'Patient': df_pat_dict, 'Encounter': df_enc_dict}

In [ ]:
########################################################################################################################
# Loop over the experiment configurations Cs and Ds, and also the granularity levels and partitions
########################################################################################################################
for conf_idx, (granular, C, D, partition) in enumerate(product(granular_list,
                                                       Cs,
                                                       Ds,
                                                       partitions), 1):
    log_head: str = f'[{conf_idx}. {granular}-level; C={C}; D={D}; partition={partition}] '
    if C == 1 and Ds.index(D) > 0:      # When C=1, all D values are the same
        continue

    ####################################################################################################################
    # Load the feature dataset created in P03a_Feature_Extraction_Patient.ipynb / P03b_Feature_Extraction_Encounter.ipynb
    ####################################################################################################################
    feat_in_path: str = os.path.join(FEAT_IN_DIR_PATH, f'{C}_encounters_{D}_days/X_{granular}_{partition}_v1.parquet')
    df: pd.DataFrame = pd.read_parquet(feat_in_path)
    print(f'{log_head}Feature dataset loaded with dimension = {df.shape}')

    ####################################################################################################################
    # Use the data dictionary to identify feature types
    #################################################################################################################### 
    df_dict: pd.DataFrame = dfs_dict[granular]
    bin_feats: list[str] = df_dict[df_dict['Variable_Type']=='Binary']['Variable_Name'].to_list()
    cont_feats: list[str] = df_dict[df_dict['Variable_Type']=='Continuous']['Variable_Name'].to_list()
    ord_feats_df: pd.DataFrame = df_dict[df_dict['Variable_Type']=='Ordinal'][['Variable_Name', 'Encoded_Values']]
    ord_feats_df['Min_Encoded'] = ord_feats_df['Encoded_Values'].apply(lambda x: min(literal_eval(x).keys()))
    ord_feats_df['Max_Encoded'] = ord_feats_df['Encoded_Values'].apply(lambda x: max(literal_eval(x).keys()))
    ord_feats_dict: dict[str, tuple[int, int]] = dict(zip(ord_feats_df['Variable_Name'], 
                                                      zip(ord_feats_df['Min_Encoded'], ord_feats_df['Max_Encoded'])))
    if 'HxSuicideAttempt30DaysPrior' in df:
        df['HxSuicideAttempt30DaysPrior'] = df['HxSuicideAttempt30DaysPrior'].replace({'Y': 1, 'N': 0}).fillna(np.nan)

    #################################################################################################################### 
    # Step 1. Clip the continuous features to mean +/- SIGMA_MULTIPLIER * standard deviations to remove outliers
    #################################################################################################################### 
    cont_feats_cur: list[str] = [c for c in cont_feats if c in df.columns]
    cont_data: np.ndarray = df[cont_feats_cur].astype(float).values
    mu: float = np.nanmean(cont_data, axis=0)
    sigma: float = np.nanstd(cont_data, axis=0)
    upper: float = mu + (SIGMA_MULTIPLIER * sigma)
    lower: float = mu - (SIGMA_MULTIPLIER * sigma)
    cont_data_clipped: np.ndarray = np.clip(cont_data, a_min=lower, a_max=upper)
    df.loc[:, cont_feats_cur] = cont_data_clipped

    # Logging
    feat_d_count, feat_d_pct, feat_v_count, feat_v_pct = diff_count(cont_data, cont_data_clipped)
    print(f'{log_head}{feat_d_count} continuous features clipped (i.e., {feat_d_pct:.2f}%)')
    print(f'{log_head}{feat_v_count} continuous values clipped (i.e., {feat_v_pct:.2f}%)')

    ####################################################################################################################
    # Step 2. Clip the ordinal features according to ord_feats_dict
    ####################################################################################################################
    ord_feats_dict_cur: dict[str, tuple[int, int]] = {k: v for k, v in ord_feats_dict.items() if k in df.columns}
    ord_data: np.ndarray = df[list(ord_feats_dict_cur.keys())].astype(float).values
    min_values: list[int] = [v[0] for v in ord_feats_dict_cur.values()]
    max_values: list[int] = [v[1] for v in ord_feats_dict_cur.values()]
    ord_data_clipped: np.ndarray = np.clip(ord_data, a_min=min_values, a_max=max_values)
    df.loc[:, list(ord_feats_dict_cur.keys())] = ord_data_clipped
    df[list(ord_feats_dict_cur.keys())] = df[list(ord_feats_dict_cur.keys())].astype('Int32')

    # Logging
    feat_d_count, feat_d_pct, feat_v_count, feat_v_pct = diff_count(ord_data, ord_data_clipped)
    print(f'{log_head}{feat_d_count} ordinal features clipped (i.e., {feat_d_pct:.2f}%)')
    print(f'{log_head}{feat_v_count} ordinal values clipped (i.e., {feat_v_pct:.2f}%)')

    ####################################################################################################################
    # Step 3. Scale the features to the unit interval using min-max normalization
    ####################################################################################################################
    scaler = MinMaxScaler()
    idx_cols: list[str] = [c for c in ['PatientDurableKey', 'EncounterKey', 'EncDate'] if c in df.columns]
    feats_cur: list[str] = [c for c in df.columns if c not in idx_cols] 
    bin_feats_cur: list[str] = [c for c in bin_feats if c in feats_cur]
    df[feats_cur] = scaler.fit_transform(df[feats_cur].astype(float).values)
    df[bin_feats_cur] = df[bin_feats_cur].astype('Int32')
    print(f'{log_head}Min-max scaling completed.')

    ####################################################################################################################
    # Step 4. Save the dataset
    ####################################################################################################################
    feat_out_path: str = feat_in_path.replace('v1.parquet', 'v2.parquet')
    df.to_parquet(feat_out_path)
    print(f'{log_head}Processed dataset saved to {feat_out_path}')
    print('-'*120)
